# Jasper Kyle Catapang - University of the Philippines Manila

# k-Nearest Neighbors Classification Algorithm Proposed Optimization
In pattern recognition, the k-nearest neighbors algorithm (k-NN) is a non-parametric method used for classification and regression. In both cases, the input consists of the k closest training examples in the feature space.

In the k-nearest neighbor algorithm (k-NN), the determination of classes for test instances is usually performed via
a majority vote system, which may ignore the similarities among data. In this research, the researcher proposes an approach to
fine-tune the selection of neighbors to be passed to the majority vote system through the construction of a random n-dimensional
hyperstructure around the test instance by introducing a new threshold parameter. The accuracy of the proposed k-NN algorithm is 85.71%, while the accuracy of the conventional k-NN algorithm is 80.95%, when performed on the Haberman’s Cancer Survival dataset—and 94.44% for the proposed k-NN algorithm, compared to the conventional’s 88.89% accuracy score on the Seeds dataset. The proposed k-NN algorithm is also on par with the conventional support vector machine algorithm accuracy, even on the Banknote Authentication and Iris datasets, even surpassing the accuracy of support vector machine on the Seeds dataset.

## Pre-processing

In [1]:
# Importing the library
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
# Importing the dataset
dataset = pd.read_csv('iris.txt')
dataset = dataset.apply(LabelEncoder().fit_transform)

In [3]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
sepal_length,150.0,15.386667,8.175743,0.0,8.0,15.0,21.0,34.0
sepal_width,150.0,9.573333,4.323103,0.0,7.0,9.0,12.0,22.0
petal_length,150.0,18.193333,11.656549,0.0,6.0,19.5,27.0,42.0
petal_width,150.0,8.993333,6.396829,0.0,2.0,9.0,14.0,21.0
class,150.0,1.000000,0.819232,0.0,0.0,1.0,2.0,2.0


In [4]:
dimensions = dataset.shape[1]
# Splitting the dataset in independent and dependent variables
X = dataset.iloc[:,:dimensions].values #first n columns are the features
y = dataset.iloc[:,-1].values #last column is the label

In [5]:
# Splitting the dataset into the training set and test set
test_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size)

In [6]:
# Feature scaling to bring the variable in a single scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(len(X_train), len(X_test))

120 30


## Model selection, k-fold cross-validation, and fitting of the conventional model
Cross-validation is commonly used in applied machine learning to compare and select a model for a given predictive modeling problem because it is easy to understand, easy to implement, and results in skill estimates that generally have a lower bias than other methods.

In [7]:
# Creating list of K for KNN
k_list = list(range(1,20,2))
k_list = [5]
# Creating list of cv scores
cv_scores = []
k_val_scores = []

# Perform 5-fold cross validation
for k in k_list:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    #print(k, y_test, y_pred)
    score = accuracy_score(y_test, y_pred)
    cv_scores.append(score)
    k_val_scores.append(k)
print("Conventional's mean accuracy: ", max(cv_scores), ", k value:", k_val_scores[cv_scores.index(max(cv_scores))])

Conventional's mean accuracy:  1.0 , k value: 5


## Elimination of training points outside generated convex-hulls 

In [8]:
def is_bounded(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    from scipy.spatial import Delaunay
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)
        simplex = hull.find_simplex(p)>=0
    return simplex

In [9]:
threshold = 35
verbose = True
i = 0
j = 0

unbounded_count = 0
unbounded_count_list = []
scores = []
mean_score = 0

orig_X_train = X_train
orig_y_train = y_train

progress = int(len(X_test)/25)
progress_list = list(range(progress, len(X_test)+1, progress))

#print(len(X_train), len(y_train))
for x in X_test:
    if verbose:
        if j in progress_list:
            print("Training sample #", j+1, "out of", len(X_test))
        elif j == len(X_test)-1:
            print("Training sample #", j+1, "out of", len(X_test))
    if threshold != 0:
        minimum = min(x)
        maximum = max(x)
        bound = np.random.random_integers(minimum-threshold, maximum+threshold, (4*(dimensions-1), dimensions))

        for xi in X_train:
            if not is_bounded(xi, bound):
                unbounded_count += 1
                #print("Unbounded: "+str(xi))
                try:
                    X_train = np.delete(X_train, i, 0)
                    y_train = np.delete(y_train, i)
                except:
                    continue
            i = i+1

    i = 0
    unbounded_count_list.append(unbounded_count)
    unbounded_count = 0
    #print(len(X_train), len(y_train))
    
    k = k_val_scores[cv_scores.index(max(cv_scores))]
    knn = KNeighborsClassifier(n_neighbors = k, n_jobs=-1)
    knn.fit(X_train, y_train)
    y_pred = knn.predict([x])
    #print(y_test[j], y_pred[0])
    if y_test[j] == y_pred[0]:
        scores.append(1.0)
    else:
        scores.append(0.0)
    
    X_train = orig_X_train
    y_train = orig_y_train
    j = j+1

mean_score = mean(scores)
print("\nProposed's mean unbounded count of neighbors: ", int(mean(unbounded_count_list)))

Training sample # 2 out of 30
Training sample # 3 out of 30
Training sample # 4 out of 30
Training sample # 5 out of 30
Training sample # 6 out of 30
Training sample # 7 out of 30
Training sample # 8 out of 30
Training sample # 9 out of 30
Training sample # 10 out of 30
Training sample # 11 out of 30
Training sample # 12 out of 30
Training sample # 13 out of 30
Training sample # 14 out of 30
Training sample # 15 out of 30
Training sample # 16 out of 30
Training sample # 17 out of 30
Training sample # 18 out of 30
Training sample # 19 out of 30
Training sample # 20 out of 30
Training sample # 21 out of 30
Training sample # 22 out of 30
Training sample # 23 out of 30
Training sample # 24 out of 30
Training sample # 25 out of 30
Training sample # 26 out of 30
Training sample # 27 out of 30
Training sample # 28 out of 30
Training sample # 29 out of 30
Training sample # 30 out of 30

Proposed's mean unbounded count of neighbors:  7


## Calculation of test set accuracy

In [10]:
print("Proposed's mean accuracy: ", mean_score, ", k value:", k)

Proposed's mean accuracy:  1.0 , k value: 5
